### CRDT Allocator
In this notebook we implement the proof-of-concept of allocator based on (address, stride, fill_map) + (size, address) model. This allocator is better suited to handle many allocations of identical size.

This is a model code, can be helpful for exprimentation, assessment and design optimization.

In [94]:
import bisect
from collections import namedtuple
import sys

In [102]:
# address / size of a single element (stride) / fill_map = bitmap indicating used and unused elements
Alloc = namedtuple("Alloc", ("address", "stride", "fill_map"))
Stripe = namedtuple("Stripe", ("size", "address"))

In [120]:
def get_alloc_size(alloc, address):
    """
    Get allocation size under a specific address or raise exception
    """
    if (address >= alloc.address) and (address - alloc.address) % alloc.stride == 0 \
        and alloc.fill_map[int((address - alloc.address) / alloc.stride)]:
        return alloc.stride
    
    raise Exception(f"Invalid address: {address}")

In [109]:
def alloc_unit(alloc: Alloc, size):
    """
    Allocates a single unit of 'size' or raises exception
    
    :param alloc: the Alloc object
    :param size: unit size
    :return: the allocated address
    """
    if size != alloc.stride:
        raise Exception("alloc_unit: bad alloc size")
    for i in range(len(alloc.fill_map)):
        if not alloc.fill_map[i]:
            alloc.fill_map[i] = True
            return alloc.address + i * size
    raise Exception("alloc_unit: out of memory")

In [113]:
def dealloc_unit(alloc: Alloc, address):
    if (address < alloc.address) or (address - alloc.address) % alloc.stride != 0:
        raise Exception(f"dealloc_unit invalid address {address}")
    unit_id = int((address - alloc.address) / alloc.stride)
    if not alloc.fill_map[unit_id]:
        raise Exception(f"dealloc_unit invalid address {address}")
    alloc.fill_map[unit_id] = False
    return any(alloc.fill_map)

In [104]:
def asize(alloc: Alloc):
    """
    Get the total size reserved by this allocation
    """
    return alloc.stride * len(alloc.fill_map)

In [116]:
class Allocator:
    # size = size of the entire available space (starting from address 0x0)
    def __init__(self, size = 1000):
        self.size = size
        # allocated addresses, sorted named tuples (address, size, fill map)
        self.allocs = []
        # blank-index, sorted tuples (size, address) - this is to enumerate available areas
        # initialize with all space available initially
        self.blanks = [Stripe(self.size, 0)]
        # list of stripes with >0 unused items
        self.stripes = []
    
    def try_alloc_from_stripe(self, size) -> int:
        """
        Try allocating from a 'stripe' of a specific size
        """
        # find stripe of exactly the size
        index = bisect.bisect_left(self.stripes, Stripe(size, 0))
        if index == len(self.stripes) or self.stripes[index].size != size:
            return None
        
        # find the corresponding alloc next
        address = self.stripes[index].address
        index = bisect.bisect_left(self.allocs, Alloc(address, size, []))
        return alloc_unit(self.allocs[index], size)
            
    def try_alloc_from_blanks(self, size, count) -> int:
        """
        Try allocating unit of 'size' and prepare for allocating count - 1 more
        
        :param size: the requested allocation size
        :param count: the requested number of identical ranges to pre-allocate
        :return allocated address or None if no sufficient blank available
        """        
        # find the 1st blank of sufficient size (i.e. >= size * count)
        index = bisect.bisect_left(self.blanks, (size * count, 0))
        if index == len(self.blanks) or index < 0:
            return None
        
        blank = self.blanks.pop(index)
        new_alloc = Alloc(blank.address, size, [False] * count)
        # take the 1st unit
        result = alloc_unit(new_alloc, size)
        # use insort to keep the allocs list sorted
        bisect.insort(self.allocs, new_alloc)
        # register the new alloc with 'stripes'
        if count > 1:        
            bisect.insort(self.stripes, Stripe(size, new_alloc.address))
        remaining = Stripe(blank.size - size * count, blank.address + size * count)
        if remaining.size > 0:
            bisect.insort(self.blanks, remaining)
        
        # address of the allocated unit
        return result
    
    def alloc(self, size) -> int:
        result = self.try_alloc_from_stripe(size)
        if result is not None:
            return result
        
        for count in [16, 1]:            
            result = self.try_alloc_from_blanks(size, count)
            if result is not None:
                return result
        
        raise Exception("out of memory")
        
    def dealloc(self, address):
        # find the allocation first
        index = bisect.bisect_left(self.allocs, Alloc(address, sys.maxsize, [])) - 1
        if index == len(self.allocs):
            raise Exception(f"Invalid address: {address}")
        
        alloc = self.allocs[index]
        if dealloc_unit(alloc, address):
            return
        
        # we need to remove the alloc entry since it's empty
        # collect the neighboring blanks
        b0, b1 = None, None
        if index > 0:
            # compare with the left-neighbor
            b0_size = self.allocs[index].address - self.allocs[index - 1].address - asize(self.allocs[index - 1])
            if b0_size > 0:
                b0 = self.allocs[index - 1].address + b0_size, b0_size
        elif alloc.address > 0:
            # 1st allocation but there's blank space left of it
            b0 = (0, alloc.address)
        
        if index < len(self.allocs) - 1:
            # compare with the right-neighbor
            b1_size = self.allocs[index + 1].address - self.allocs[index].address - asize(self.allocs[index])
            if b1_size > 0:
                b1 = self.allocs[index + 1].address - b1_size, b1_size
        elif alloc.address + asize(alloc) < self.size:
            # last allocation but there's blank space right of it
            b1 = (alloc.address + asize(alloc), self.size - alloc.address - asize(alloc))
        
        # remove blanks
        if b0 is not None:
            del self.blanks[bisect.bisect_left(self.blanks, b0)]
        if b1 is not None: 
            del self.blanks[bisect.bisect_left(self.blanks, b1)]
        
        # remove the allocation
        del self.allocs[index]
        
        # re-insert a merged blank
        b0 = alloc if b0 is None else b0
        b1 = alloc if b1 is None else b1
        blank = Stripe(b0[0], b1[0] + b1[1] - b0[0])
        bisect.insort(self.blanks, blank)
    
    def get_alloc_size(self, address):
        index = bisect.bisect_left(self.allocs, Alloc(address, sys.maxsize, [])) - 1        
        if index == len(self.allocs) or index < 0:
            raise Exception(f"Invalid address: {address}")
            
        return get_alloc_size(self.allocs[index], address)
    
    def __repr__(self):
        return f"blanks:{'+'.join([str(b) for b in self.blanks])}"

In [115]:
def test_allocs_only():
    a = Allocator()
    my_objects = []
    # 3 allocations of size '3'
    for _ in range(3):
        my_objects.append(a.alloc(3))
    
    print(f"my_objects: {my_objects}")
    print(a)

In [107]:
def test_allocs_dealloc():
    a = Allocator()
    my_objects = []
    # 3 allocations of size '3'
    for _ in range(3):
        my_objects.append(a.alloc(3))
    
    a.dealloc(my_objects.pop(1))
    a.dealloc(my_objects.pop(1))
    a.dealloc(my_objects.pop(0))    
    print(f"my_objects: {my_objects}")
    print(a)

In [118]:
def test_get_alloc_size():
    a = Allocator()
    my_objects = []
    # 3 allocations of size '3'
    for _ in range(3):
        my_objects.append(a.alloc(3))
    # 3 allocations of size '12'
    for _ in range(3):
        my_objects.append(a.alloc(12))
    
    print(my_objects)
    print(a.get_alloc_size(my_objects[4]))
    print(a.get_alloc_size(my_objects[0]))

In [121]:
test_get_alloc_size()

[0, 3, 6, 48, 60, 72]
12
3
